# Data structures and geometries

In [ ]:
import sys
sys.path.append("../../cuqipy")
import cuqi
import numpy as np
from cuqi.samples import CUQIarray
from cuqi.samples import Samples

# Fundamental data structure is CUQIarray:

In [ ]:
v = CUQIarray([0,1,2,3,4,5,6,7,8])

In [ ]:
v

### TODO Should a default geometry be assigned to allow plotting? Gives error currently

In [ ]:
v.plot()

## Being subclassed from NumPy array, all NumPy operations can be used directly on CUQIarrays

### Algebra

In [ ]:
2*v +3 - np.sqrt(v) + np.sin(np.pi*v)*np.exp(-0.5*v)

### Logical operations, logical indexing and assignment:

In [ ]:
v[v < 5] = 0

In [ ]:
v

### TODO reduction operations such as .min() produce an array of different size than geometry. Do we need to do something about that?

In [ ]:
v.min()

### CUQI array can be equipped with geometry

In [ ]:
geo = cuqi.geometry._DefaultGeometry(v.__len__())

In [ ]:
v.geometry = geo

In [ ]:
v

### CUQIarray has a plot method

In [ ]:
v.plot()

# Geometry

### List which geometries are available:
- DefaultGeometry
- Continuous1D
- Contionous2D
- Image2D
- Discrete
- MappedGeometry
- ? WrappedGeometry
- KLExpansion
- KLExpansion_full

### If changing the geometry, the plot method will be updated:

In [ ]:
v.geometry = cuqi.geometry.Continuous2D((3,3))

In [ ]:
v.plot()

### Parameters and function values

In [ ]:
v

In [ ]:
v.is_par

In [ ]:
v.shape

In [ ]:
vf = v.funvals

In [ ]:
vf

In [ ]:
vf.is_par

### TODO Is this shape correct?  Will be changed to par_shape?

In [ ]:
vf.shape

In [ ]:
vf.parameters

### How/whether to get to par2fun, fun2par

In [ ]:
geo.par2fun(v)

In [ ]:
geo.fun2par(vf)

#### TODO would a MappedGeometry or something else show the difference more clearly

## A bit strange to add parameter and funvals versions. The order determines whether is_par true or false:

In [ ]:
vf + v

# Samples object
### Describe Samples object here as CUQIarray with multiple data points/list of CUQIarrays?

In [ ]:
s = Samples(np.random.rand(9, 100), geometry=cuqi.geometry.Image2D((3,3)))

## Printing, repr of Samples is not very informative

In [ ]:
print(s)

In [ ]:
s.geometry

## Similar to CUQIarray can plot selected samples, to be given as list/numpy array, using the plotting method given by geometry

In [ ]:
s.plot([0,10,20,30,40])

## TODO What else to do with Samples object, that is not yet analyzing actual samples, like plot_ci

Cannot slice Samples, would expect to get CUQIarray.

In [ ]:
s[10,:]

# From Exercise02 on geometries

In [ ]:
true_mu = np.array([5, 3, 1])
Z = cuqi.distribution.Gaussian(mean=true_mu, std=true_mu)

By default no particular structure or space is assumed of the parameters. If we want to express that parameters constitute for example a 2D image or are a set of discrete named parameters we can specify this by means of a CUQIpy geometry. 

By default distributions contain a default (trivial) geometry.

In [ ]:
Z.geometry

We may equip the distribution with a different geometry, either when creating it, or afterwards. For example if the five parameters represent labelled quantities such as height, width, depth, weight and density we can use a `Discrete` geometry:

In [ ]:
geom = cuqi.geometry.Discrete(['height','width','depth'])

We can update the distribution's geometry and generate some new samples:

In [ ]:
Z.geometry = geom

In [ ]:
sZ2 = Z.sample(100)

The samples will now know about their new `Discrete` geometry and the plotting style will be changed:

In [ ]:
sZ2.plot();

The credibility interval plot style is also updated to show errorbars for the `Discrete` geometry:

In [ ]:
sZ2.plot_ci(95, exact=true_mu)

And the similarly in the chain plot the legend reflects the particular labels:

In [ ]:
sZ2.plot_chain([0,2])

Another use of geometry is to represent 1D or 2D versions of the same distribution (prior). A Gaussian Markov Random Field (GMRF) can be used in 1 or 2 spatial dimensions, which is represented using `Continuous1D` and `Continuous2D` geometries:

In [ ]:
N = 100     # number of pixels
dom = 2     # 1D or 2D domain

x = np.linspace(0,1,N)

if (dom == 1):
    geometry = cuqi.geometry.Continuous1D(x)
elif (dom == 2):
    geometry = cuqi.geometry.Continuous2D((x, x))

In this example in 1D there will be N parameters and in 2D there will be N^2 parameters. We can check the number of parameters of the geometry as well as its type:

In [ ]:
geometry.dim

In [ ]:
type(geometry)

We can now specify a GMRF distribution (with some chosen mean, precision, boundary conditions etc.) The same exact code will work in 1D and 2D due to the geometry:

In [ ]:
cuqi.distribution.GaussianPrec?

In [ ]:
mean = np.zeros(geometry.dim)
prec = 4
pX = cuqi.distribution.GMRF(mean, prec, N, physical_dim=dom, bc_type='zero', geometry=geometry)
pX = cuqi.distribution.GaussianCov(np.zeros(geometry.dim), cov=2, geometry=geometry)

With the distribution set up, we are ready to generate some samples

In [ ]:
# call method to sample
sampleX = pX.sample(50)

In [ ]:
sampleX.shape

We plot a couple of samples:

In [ ]:
sampleX.plot()   

#### Try yourself (optional):  
 - Go back and change `dom` to 2 to get the 2D case and rerun the subsequent cells.
 - Play with the number of pixels `N` as well as parameters of the GMRF and see the effect on the samples.